<a href="https://colab.research.google.com/github/recinilt/kodlama/blob/main/YoutubeVideoTranscript2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # 📝 YouTube Video Transkript Oluşturucu (GPU Destekli)

import torch

def gpu_baglanti_kontrol():
    # GPU bağlantısını kontrol et
    if torch.cuda.is_available():
        gpu_tipi = torch.cuda.get_device_name(0)
        print(f"Zaten bir GPU'ya bağlısınız: {gpu_tipi}")
        return True
    else:
        print("GPU bağlantısı bulunamadı. T4 GPU'ya bağlanmaya çalışılıyor...")
        return False

def t4_gpu_baglan():
    try:
        # GPU bağlı değilse runtime'ı değiştirmek için
        from google.colab import runtime
        runtime.change_runtime('accelerator', 'GPU')
        print("T4 GPU'ya bağlanma işlemi başlatıldı. Notebook yeniden başlatılacak.")
        return True
    except:
        print("GPU bağlantısı yapılamadı. Manuel olarak 'Çalışma Zamanı > Çalışma zamanı türünü değiştir' menüsünden GPU seçeneğini etkinleştirin.")
        return False

# GPU bağlantısını kontrol et ve gerekirse T4 GPU'ya bağlan
if not gpu_baglanti_kontrol():
    t4_gpu_baglan()



import os
import sys
from IPython.display import HTML, display
from IPython import get_ipython

# GPU durumunu kontrol et
gpu_var_mi = os.system('nvidia-smi > /dev/null 2>&1')

if gpu_var_mi != 0:
    # GPU bağlı değilse uyarı göster ve notebook'u durdur
    display(HTML('''
    <div style="background-color: #f8d7da; color: #721c24; padding: 15px; border: 1px solid #f5c6cb; border-radius: 5px; margin: 10px 0; font-size: 16px;">
        <h3 style="color: #721c24;">❌ GPU BAĞLANTISI YOK!</h3>
        <p><b>Bu notebook GPU gerektirir. Lütfen şu adımları izleyin:</b></p>
        <ol>
            <li>Üst menüden <b>Çalışma Zamanı</b> > <b>Çalışma zamanı türünü değiştir</b>'e tıklayın</li>
            <li>'Donanım hızlandırıcı' altından <b>T4 GPU</b>'yu seçin</li>
            <li><b>Kaydet</b>'e tıklayın ve notebook yeniden başlatıldığında kodu tekrar çalıştırın</li>
        </ol>
    </div>
    '''))

    # Tüm hücrelerin çalışmasını durdur - daha agresif yaklaşım
    get_ipython().kernel.do_shutdown(True)
    sys.exit("GPU bağlantısı olmadığı için notebook durduruldu.")
else:
    print("✅ GPU bağlantısı mevcut. Notebook çalışmaya devam ediyor.")

print("Gerekli kütüphaneler yükleniyor...")
!pip install -q openai-whisper
!pip install -q ffmpeg-python
!apt-get -qq install ffmpeg
print("Gerekli kütüphaneler yüklendi")
import os
import re
import time
import glob
import torch
import whisper
import base64
import subprocess
from pathlib import Path
from datetime import timedelta
from google.colab import files
import IPython.display as display
import gc  # Garbage collector

# Gerekli kütüphaneleri yükle
!pip install -q openai-whisper yt-dlp
!apt-get -qq install ffmpeg



print("# YouTube veya Yerel Video Transkript Oluşturucu")
print("Bu kodla YouTube videoları veya yerel dosyaları otomatik olarak işleyip transkript oluşturabilirsiniz.\n")
print("📌 Not: YouTube videoları için en düşük görüntü kalitesi + en iyi ses kalitesi kullanılacak (daha hızlı indirme)")

#@markdown ## Video Kaynağı Seçimi ve Ayarlar

#@markdown ### Video Kaynağı
video_source = "YouTube Linki" #@param ["YouTube Linki", "Yerel Dosya"]

#@markdown ### YouTube URL (YouTube seçiliyse doldurulmalı)
youtube_url = "https://www.youtube.com/watch?v=9jJt-ndMfVk&t=3s" #@param {type:"string"}

#@markdown ### Transkript Ayarları
language = "tr" #@param ["tr", "en", "de", "fr", "es", "it", "ru", "ar", "zh", "ja", "ko", "pt"]
model_size = "large" #@param ["small", "medium", "large"]
high_quality = True #@param {type:"boolean"}
timestamp_output = True #@param {type:"boolean"}

#@markdown ### Video Bölme Ayarları (Uzun Videolar İçin)
segment_length_minutes = 2 #@param {type:"slider", min:2, max:30, step:2}
delete_segments_after = True #@param {type:"boolean"}

# YouTube URL işleme ve ID çıkarma fonksiyonu
def extract_youtube_id(url):
  """YouTube URL'sinden video ID'sini çıkarır"""
  # & işareti varsa ilk kısmı al
  if '&' in url:
    url = url.split('&')[0]

  # Standart YouTube URL formatları
  patterns = [
    r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/watch\?v=([^\/\?\&]+)',  # youtube.com/watch?v=ID
    r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/embed\/([^\/\?\&]+)',    # youtube.com/embed/ID
    r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/v\/([^\/\?\&]+)',        # youtube.com/v/ID
    r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/shorts\/([^\/\?\&]+)',   # youtube.com/shorts/ID
    r'(?:https?:\/\/)?(?:www\.)?youtu\.be\/([^\/\?\&]+)'               # youtu.be/ID
  ]

  # Her formatı dene
  for pattern in patterns:
    match = re.search(pattern, url)
    if match:
      return match.group(1)

  # Hiçbir format eşleşmezse None döndür
  return None

def download_youtube_video(youtube_url, output_path=None):
  """YouTube videosunu indirir (yt-dlp kullanarak) - Ses odaklı, düşük görüntü kalitesi"""

  # Video ID'sini çıkar
  video_id = extract_youtube_id(youtube_url)
  if not video_id:
    raise ValueError(f"Geçerli bir YouTube URL'si değil: {youtube_url}")

  print(f"YouTube video ID: {video_id}")
  print(f"Video indiriliyor (ses odaklı, düşük görüntü kalitesi)...")

  # Çıktı dosya adını belirle
  if output_path is None:
    #output_path = f"youtube_video_{video_id}.mp4"
    output_path = f"youtube_video_{video_id}.mp3"

  # yt-dlp kullanarak en düşük görüntü kalitesi + en iyi ses kalitesinde videoyu indir
  #cmd = [
  #    'yt-dlp',
  #    '-f', 'worstvideo[ext=mp4]+bestaudio[ext=m4a]/worst[ext=mp4]/worst',
  #    '-o', output_path,
  #    youtube_url,
  #    '--no-warnings',
  #    '--quiet'
  #]

  cmd = [
    'yt-dlp',
    '-f', 'bestaudio[ext=m4a]/bestaudio',  # Sadece en iyi ses kanalını indir
    '-o', output_path.replace('.mp4', '.m4a'),  # Uzantıyı .m4a olarak değiştir
    youtube_url,
    '--no-warnings',
    '--quiet',
    '--extract-audio',  # Sadece ses kanalını çıkar
    '--audio-format', 'mp3',  # MP3 formatına dönüştür
    '--audio-quality', '0',  # En yüksek ses kalitesi
    '--force-ipv4',  # IPv4 kullan (bazen daha hızlı)
    '--concurrent-fragments', '4'  # Paralel indirme parçaları
]

  try:
    subprocess.run(cmd, check=True)

    # Video süresini al
    duration_cmd = [
        'ffprobe',
        '-v', 'error',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        output_path
    ]
    duration = float(subprocess.check_output(duration_cmd).decode('utf-8').strip())

    # Video bilgilerini göster
    print(f"Video başarıyla indirildi: {output_path}")
    print(f"Video uzunluğu: {duration:.1f} saniye ({format_time_duration(duration)})")

    return output_path
  except subprocess.CalledProcessError as e:
    raise Exception(f"Video indirme hatası: {str(e)}")

def format_time(seconds):
  """Saniye cinsinden zamanı SS:DD:SS formatına dönüştürür"""
  hours = int(seconds // 3600)
  minutes = int((seconds % 3600) // 60)
  seconds = int(seconds % 60)
  return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def format_time_duration(seconds):
  """Saniye cinsinden süreyi okunabilir formata dönüştürür"""
  hours = int(seconds // 3600)
  minutes = int((seconds % 3600) // 60)
  secs = int(seconds % 60)

  if hours > 0:
    return f"{hours} saat {minutes} dakika {secs} saniye"
  elif minutes > 0:
    return f"{minutes} dakika {secs} saniye"
  else:
    return f"{secs} saniye"

def download_link(path, filename):
  """Dosya indirme linki oluşturur"""
  with open(path, 'r', encoding='utf-8') as f:
    content = f.read()

  b64 = base64.b64encode(content.encode('utf-8')).decode()

  download_link_html = f'''
  <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin: 10px 0;">
    <p style="font-weight: bold; margin-bottom: 10px;">Transkript başarıyla oluşturuldu!</p>
    <a href="data:text/plain;base64,{b64}" download="{filename}"
       style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none;
              border-radius: 4px; display: inline-block; margin-top: 5px;">
      Transkript Dosyasını İndir
    </a>
  </div>
  '''

  display.display(display.HTML(download_link_html))

def get_video_duration(input_file):
  """Video süresini saniye cinsinden döndürür"""
  cmd = [
      'ffprobe',
      '-v', 'error',
      '-show_entries', 'format=duration',
      '-of', 'default=noprint_wrappers=1:nokey=1',
      input_file
  ]
  output = subprocess.check_output(cmd).decode('utf-8').strip()
  return float(output)

# Eksik olan optimize_for_whisper fonksiyonunu ekleyelim
def optimize_for_whisper(video_path):
    audio_path = f"{Path(video_path).stem}_audio.mp3"

    cmd = [
        'ffmpeg',
        '-i', video_path,
        '-vn',
        '-c:a', 'mp3',
        '-b:a', '192k',  # 128k yerine 192k kullan
        '-ar', '16000',
        '-ac', '1',
        '-af', 'highpass=f=200,lowpass=f=3000,volume=2',  # Ses filtreleri ekle
        '-y',
        audio_path
    ]

    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return audio_path

def split_video_ffmpeg(input_file, segment_length=15, output_dir=None):
  """
  Video veya ses dosyasını FFmpeg kullanarak hızlıca belirtilen dakikalık segmentlere böler

  Args:
    input_file: Kaynak video veya ses dosyası
    segment_length: Her segmentin dakika cinsinden uzunluğu
    output_dir: Çıktı dizini (None ise input_file'ın bulunduğu dizin kullanılır)

  Returns:
    Oluşturulan parça dosyalarının listesi
  """
  if output_dir is None:
    output_dir = os.path.dirname(input_file)
    if output_dir == '':
      output_dir = '.'

  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  # Dosya süresini al
  print(f"Dosya inceleniyor: {input_file}")
  total_duration = get_video_duration(input_file)

  # Segment sayısını hesapla
  segment_length_sec = segment_length * 60
  num_segments = int(total_duration / segment_length_sec) + (1 if total_duration % segment_length_sec > 0 else 0)

  print(f"Toplam süre: {total_duration/60:.1f} dakika ({format_time_duration(total_duration)})")
  print(f"Dosya {num_segments} parçaya bölünecek ({segment_length} dakikalık dilimler)...")

  # Çıktı dosya listesi
  output_files = []

  # Dosya uzantısını al - aynı uzantıyı çıktılar için kullan
  file_ext = os.path.splitext(input_file)[1]
  if not file_ext:
    file_ext = ".mp3"  # Varsayılan uzantı

  # Her segment için
  for i in range(num_segments):
    # Segment başlangıç ve bitiş zamanı
    start_time = i * segment_length_sec
    end_time = min((i + 1) * segment_length_sec, total_duration)

    # Dosya adı
    base_name = os.path.splitext(os.path.basename(input_file))[0]
    output_file = os.path.join(output_dir, f"{base_name}_part{i+1:02d}{file_ext}")

    # FFmpeg ile segmenti kes
    print(f"Bölüm {i+1}/{num_segments} kesiliyor ({format_time(start_time)} - {format_time(end_time)})...")

    # FFmpeg komutu - stream kopyalama modu (-c copy) ile hızlı kesme
    cmd = [
        'ffmpeg',
        '-i', input_file,
        '-ss', str(start_time),
        '-to', str(end_time),
        '-c', 'copy',  # Stream kopyalama - yeniden kodlama yok!
        '-y',  # Mevcut dosyaların üzerine yaz
        output_file
    ]

    # Komutu çalıştır
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    output_files.append(output_file)

  print(f"Bölme işlemi tamamlandı. {len(output_files)} parça oluşturuldu.")
  return output_files

def transcribe_segment(audio_path, model, language="tr", high_quality=True, timestamp_output=True):
  """Tek bir ses segmentinin transkriptini oluşturur"""

  segment_start_time = time.time()
  print(f"Segment işleniyor: {os.path.basename(audio_path)}")

  try:
    # Transkript ayarları
    options = {
      "language": language,
      "task": "transcribe",
      "verbose": False,
    }

    # Yüksek kalite için ek parametreler
    if high_quality:
      options.update({
        "beam_size": 8,
        "best_of": 8,
        "temperature": [0.0, 0.2, 0.4, 0.6],
        "compression_ratio_threshold": 2.0,
        "condition_on_previous_text": True,
        "fp16": False  # Yüksek kalite için FP32 kullan
      })

    # Bellek temizliği
    gc.collect()
    torch.cuda.empty_cache()

    # Transkript oluştur
    result = model.transcribe(audio_path, **options)

    # Transkript metnini al
    transcript_text = result["text"]

    # Çıktı dosya adını belirle
    output_path = f"{Path(audio_path).stem}_transkript.txt"

    # Transkripti dosyaya kaydet
    with open(output_path, "w", encoding="utf-8") as f:
      f.write(transcript_text)

      # Eğer segments bilgisi varsa ve zaman damgası istendiyse
      if timestamp_output and "segments" in result:
        f.write("\n\n## ZAMANLI TRANSKRİPT ##\n\n")
        for segment in result["segments"]:
          segment_start = segment["start"]
          segment_end = segment["end"]
          text = segment["text"]
          timestamp = f"[{format_time(segment_start)} --> {format_time(segment_end)}]"
          f.write(f"{timestamp} {text}\n")

    # İşlem süresini hesapla
    segment_elapsed_time = time.time() - segment_start_time
    print(f"Segment tamamlandı: {os.path.basename(output_path)}")
    print(f"İşlem süresi: {segment_elapsed_time:.2f} saniye ({format_time_duration(segment_elapsed_time)})")

    # Orijinal result'ı bellekten temizlemek için sadece ihtiyacımız olan verileri saklayalım
    segments_copy = None
    if "segments" in result:
      # Segment verilerinin hafif bir kopyasını oluşturalım
      segments_copy = []
      for segment in result["segments"]:
        segments_copy.append({
          "start": segment["start"],
          "end": segment["end"],
          "text": segment["text"]
        })

    # Artık ihtiyacımız olmayan result verisini temizleyelim
    result = None

    return {
      "path": output_path,
      "text": transcript_text,
      "segments": segments_copy
    }
  except Exception as e:
    print(f"❌ Segment işleme hatası: {str(e)}")
    print(f"Hatalı dosya: {audio_path}")
    import traceback
    traceback.print_exc()
    return {
      "path": f"{Path(audio_path).stem}_hata.txt",
      "text": f"Transkript oluşturulamadı: {str(e)}",
      "segments": None
    }

def merge_transcripts(transcript_results, input_file, timestamp_output=True):
  """Tüm transkript sonuçlarını birleştirir ve bir dosyaya kaydeder"""

  # Ana video dosya adını al
  base_name = os.path.splitext(os.path.basename(input_file))[0]
  full_transcript_path = f"{base_name}_tam_transkript.txt"

  # Tüm metinleri birleştir
  full_text = ""
  all_segments = []
  segment_time_offset = 0

  # Her segment için
  for i, result in enumerate(transcript_results):
    # Tam metine bölüm bilgisini ve transkripti ekle
    full_text += f"\n\n--- BÖLÜM {i+1} ---\n\n"
    full_text += result["text"]

    # Zaman damgalı segmentleri ayarla
    if timestamp_output and result["segments"] is not None:
      # Her segmentin zaman bilgisini orijinal videodaki konumuna göre ayarla
      for segment in result["segments"]:
        # Yeni segment oluştur ve zaman bilgisini güncelle
        adjusted_segment = segment.copy()
        adjusted_segment["start"] += segment_time_offset
        adjusted_segment["end"] += segment_time_offset
        all_segments.append(adjusted_segment)

      # Bir sonraki bölüm için zaman farkını güncelle
      if result["segments"]:
        last_segment = result["segments"][-1]
        segment_time_offset += last_segment["end"]

  # Tam transkripti dosyaya kaydet
  with open(full_transcript_path, "w", encoding="utf-8") as f:
    # Önce tam metni yaz
    f.write(full_text)

    # Eğer zaman damgalı çıktı isteniyorsa ve segmentler varsa
    if timestamp_output and all_segments:
      # Zaman damgalı transkripti ekle
      f.write("\n\n## TAM ZAMANLI TRANSKRİPT ##\n\n")
      for segment in all_segments:
        start_time = segment["start"]
        end_time = segment["end"]
        text = segment["text"]
        timestamp = f"[{format_time(start_time)} --> {format_time(end_time)}]"
        f.write(f"{timestamp} {text}\n")

  print(f"Tam transkript oluşturuldu: {full_transcript_path}")
  return full_transcript_path

# Ana işlev
def process_video_file(video_path, language="tr", model_size="large", high_quality=True,
                        timestamp_output=True, segment_length_minutes=15, delete_segments_after=True):
  """
  Yerel veya indirilen videoyu işleyip transkript oluşturur
  """
  try:
    total_start_time = time.time()

    # Sadece ses kanalına dönüştür (optimize)
    print("\n--- 1. SES KANALI OPTİMİZASYONU ---")
    audio_path = optimize_for_whisper(video_path)
    print(f"Video ses kanalı çıkarıldı: {audio_path}")

    # 2. Ses dosyasını parçalara böl
    print("\n--- 2. SES BÖLME (FFmpeg ile Hızlı Kesim) ---")
    audio_segments = split_video_ffmpeg(audio_path, segment_length=segment_length_minutes)

    # 3. GPU kontrolü
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cuda":
      print(f"\nGPU kullanılıyor: {torch.cuda.get_device_name(0)}")
    else:
      print("\nUYARI: GPU bulunamadı! CPU kullanılacak (çok yavaş olabilir)")
      print("Large model için GPU önerilir. Colab > Çalışma zamanı > Çalışma zamanı türünü değiştir > GPU seçeneğini kullanın")

    # 4. Whisper modelini yükle
    print(f"\n--- 3. MODEL YÜKLEME ---")
    print(f"{model_size} boyutunda model yükleniyor...")

    # Belleği temizle
    gc.collect()
    if device == "cuda":
      torch.cuda.empty_cache()

    # Modeli yükle
    #model = whisper.load_model(model_size, device=device)
    model = whisper.load_model(model_size, device=device, download_root="/tmp")
    print(f"Model yüklendi")

    # 5. Her ses parçasını işle
    print(f"\n--- 4. TRANSKRİPT OLUŞTURMA ---")
    transcript_results = []

    for i, segment_path in enumerate(audio_segments):
      print(f"\nSes parçası {i+1}/{len(audio_segments)} işleniyor...")

      # Transkript oluştur
      result = transcribe_segment(
        segment_path,
        model,
        language=language,
        high_quality=high_quality,
        timestamp_output=timestamp_output
      )

      # Sonucu listeye ekle
      transcript_results.append(result)

      # Segment işlendikten sonra belleği temizle
      print("Bellek temizleniyor...")
      gc.collect()
      torch.cuda.empty_cache()
      print("Bellek temizlendi.")

    # 6. Tüm transkriptleri birleştir
    print(f"\n--- 5. TRANSKRİPTLERİ BİRLEŞTİRME ---")
    full_transcript_path = merge_transcripts(
      transcript_results,
      video_path,
      timestamp_output=timestamp_output
    )

    # 7. Dosya indirme bağlantısı oluştur
    print(f"\n--- İŞLEM TAMAMLANDI ---")
    download_link(full_transcript_path, os.path.basename(full_transcript_path))

    # 8. Geçici dosyaları temizle (isteğe bağlı)
    if delete_segments_after:
      print("\nGeçici dosyalar temizleniyor...")
      # Ses parçalarını temizle
      for segment_path in audio_segments:
        if os.path.exists(segment_path):
          os.remove(segment_path)
        # Segment transkript dosyasını da temizle
        transcript_path = f"{Path(segment_path).stem}_transkript.txt"
        if os.path.exists(transcript_path):
          os.remove(transcript_path)

      # Ana ses dosyasını temizle
      if os.path.exists(audio_path) and audio_path != video_path:  # Orijinal video dosyasını silmeyelim
        os.remove(audio_path)

      print("Geçici dosyalar temizlendi")

    # Toplam süreyi hesapla
    total_elapsed_time = time.time() - total_start_time
    print(f"\nToplam işlem süresi: {total_elapsed_time:.2f} saniye ({format_time_duration(total_elapsed_time)})")

    # Önizleme
    print("\nTRANSKRİPT ÖN İZLEME (ilk 500 karakter):")
    print("=" * 80)
    with open(full_transcript_path, 'r', encoding='utf-8') as f:
      preview = f.read(500) + "..."
      print(preview)
    print("=" * 80)

    return full_transcript_path

  except Exception as e:
    print(f"\n❌ Hata oluştu: {str(e)}")
    import traceback
    traceback.print_exc()
    return None

# Ana işlem bloğu
try:
  if video_source == "YouTube Linki":
    if not youtube_url or youtube_url.strip() == "":
      print("❌ Lütfen bir YouTube URL'si girin!")
    else:
      # YouTube video ID'sini çıkar
      video_id = extract_youtube_id(youtube_url)
      if not video_id:
        print(f"❌ Geçerli bir YouTube URL'si değil: {youtube_url}")
      else:
        # YouTube videosunu indir
        video_path = download_youtube_video(youtube_url)

        # Videoyu işle
        process_video_file(
          video_path,
          language=language,
          model_size=model_size,
          high_quality=high_quality,
          timestamp_output=timestamp_output,
          segment_length_minutes=segment_length_minutes,
          delete_segments_after=delete_segments_after
        )
  else:  # Yerel Dosya
    # Dosya yükleme
    uploaded_files = files.upload()

    if uploaded_files:
      uploaded_file_path = list(uploaded_files.keys())[0]
      print(f"✓ Video yüklendi: {uploaded_file_path}")

      # Videoyu işle
      process_video_file(
        uploaded_file_path,
        language=language,
        model_size=model_size,
        high_quality=high_quality,
        timestamp_output=timestamp_output,
        segment_length_minutes=segment_length_minutes,
        delete_segments_after=delete_segments_after
      )
    else:
      print("❌ Dosya yükleme iptal edildi")
except Exception as e:
  print(f"❌ Genel bir hata oluştu: {str(e)}")
  import traceback
  traceback.print_exc()

Zaten bir GPU'ya bağlısınız: Tesla T4
✅ GPU bağlantısı mevcut. Notebook çalışmaya devam ediyor.
Gerekli kütüphaneler yükleniyor...
Gerekli kütüphaneler yüklendi
# YouTube veya Yerel Video Transkript Oluşturucu
Bu kodla YouTube videoları veya yerel dosyaları otomatik olarak işleyip transkript oluşturabilirsiniz.

📌 Not: YouTube videoları için en düşük görüntü kalitesi + en iyi ses kalitesi kullanılacak (daha hızlı indirme)
YouTube video ID: 9jJt-ndMfVk
Video indiriliyor (ses odaklı, düşük görüntü kalitesi)...
Video başarıyla indirildi: youtube_video_9jJt-ndMfVk.mp3
Video uzunluğu: 8036.2 saniye (2 saat 13 dakika 56 saniye)

--- 1. SES KANALI OPTİMİZASYONU ---
Video ses kanalı çıkarıldı: youtube_video_9jJt-ndMfVk_audio.mp3

--- 2. SES BÖLME (FFmpeg ile Hızlı Kesim) ---
Dosya inceleniyor: youtube_video_9jJt-ndMfVk_audio.mp3
Toplam süre: 133.9 dakika (2 saat 13 dakika 56 saniye)
Dosya 67 parçaya bölünecek (2 dakikalık dilimler)...
Bölüm 1/67 kesiliyor (00:00:00 - 00:02:00)...
Bölüm 2/67 ke

100%|██████████| 11999/11999 [00:45<00:00, 262.11frames/s]


Segment tamamlandı: youtube_video_9jJt-ndMfVk_audio_part01_transkript.txt
İşlem süresi: 46.36 saniye (46 saniye)
Bellek temizleniyor...
Bellek temizlendi.

Ses parçası 2/67 işleniyor...
Segment işleniyor: youtube_video_9jJt-ndMfVk_audio_part02.mp3


100%|██████████| 11995/11995 [01:06<00:00, 180.76frames/s]


Segment tamamlandı: youtube_video_9jJt-ndMfVk_audio_part02_transkript.txt
İşlem süresi: 66.73 saniye (1 dakika 6 saniye)
Bellek temizleniyor...
Bellek temizlendi.

Ses parçası 3/67 işleniyor...
Segment işleniyor: youtube_video_9jJt-ndMfVk_audio_part03.mp3


...

100%|██████████| 11995/11995 [02:53<00:00, 69.11frames/s]


Segment tamamlandı: youtube_video_9jJt-ndMfVk_audio_part35_transkript.txt
İşlem süresi: 173.89 saniye (2 dakika 53 saniye)
Bellek temizleniyor...
Bellek temizlendi.

Ses parçası 36/67 işleniyor...
Segment işleniyor: youtube_video_9jJt-ndMfVk_audio_part36.mp3


100%|██████████| 11995/11995 [01:33<00:00, 127.80frames/s]


Segment tamamlandı: youtube_video_9jJt-ndMfVk_audio_part36_transkript.txt
İşlem süresi: 94.21 saniye (1 dakika 34 saniye)
Bellek temizleniyor...
Bellek temizlendi.

Ses parçası 37/67 işleniyor...
Segment işleniyor: youtube_video_9jJt-ndMfVk_audio_part37.mp3


 97%|█████████▋| 11634/11999 [00:52<00:01, 212.32frames/s]